[View in Colaboratory](https://colab.research.google.com/github/Manodhayan/dHaYaN/blob/master/Exam_Preparation_Online_Intern_Selection_Assignment.ipynb)

# Exam Preparation Online Intern Selection Assignment

### References


In [0]:
# https://gist.github.com/jmcarp/7105045
# https://github.com/shakkaist/Python/blob/master/Day2Session2/pdfconverter.py
# https://www.geeksforgeeks.org/working-with-pdf-files-in-python/
# https://github.com/csurfer/rake-nltk

### Installing Required Packages

In [0]:
!pip install PyPDF2
!pip install rake-nltk
!pip install summa
import nltk
nltk.download('stopwords')

### Accessing Data from GDrive

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_list = drive.ListFile({'q': "'1mLNJza6w4ywUAJpHhKzucHqzJbQgoNL9' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))
  
pdf_data = drive.CreateFile({'id': '1Sg0Q1gtmXBrS9aYcKCm9R0GfvXCG0Nv_'})
pdf_data.GetContentFile("java_book.pdf")

## PDF to Text Conversion

In [0]:
def pagewise_text_extraction(pdfFileObj,Num_of_Pages,documented_text):
  # Footer Words
  footer_words=['Java Basics','1996-2003 jGuru.com. All Rights Reserved.','-','1','2','3','4','5','6','7','8','9','10','©']
  
  #Get rid of last page(This doesn't suit for all documents) since it doesn't have any valid information
  Num_of_Pages=Num_of_Pages-1
  # Get text from the pdf
  
  for page in range(Num_of_Pages):
    pageObj = pdfReader.getPage(page)

    #Enocding into ascii format
    rough_text=pageObj.extractText().encode('ascii', 'ignore')
 
    # Removing Page Header,Footer and Page numbers
    for words in footer_words:
      cleaned_text=rough_text.replace(words,'')
      rough_text=cleaned_text

    documented_text=documented_text+cleaned_text
  return documented_text
    
    

In [0]:
import PyPDF2

documented_text=''

#Opening the PDF file
pdfFileObj = open('java_book.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
documented_text=pagewise_text_extraction(pdfFileObj,pdfReader.numPages,documented_text)
 
# closing the pdf file object
pdfFileObj.close()

documented_text

## Cleaning Text

In [0]:
from nltk.corpus import stopwords
# Extracting meaningfull words
def clean_text(text):
        stopwords_set = set(stopwords.words("english"))
        filtered_words = [e.lower() for e in text.split() if len(e) >= 3]
        stopwords_less_words = [word for word in filtered_words if not word in stopwords_set]
        return ' '.join(stopwords_less_words)

In [0]:
documented_text=clean_text(documented_text)
print(documented_text)
usefull_text=''

#Removing Special charectors
import re
for words in documented_text.split():
  words = re.sub(r'[^A-Za-z]', ' ', words)
  usefull_text=usefull_text+' '+words
  #print(usefull_text)
  #print(words)
print(usefull_text)

## Keyword Extraction

In [0]:
#Performing Keyword Extraction

from rake_nltk import Rake
rake =Rake(min_length=2, max_length=5)

rake.extract_keywords_from_text(usefull_text)

# To get keyword phrases ranked highest to lowest with scores.
ranked_text=rake.get_ranked_phrases_with_scores()

In [310]:
import pandas as pd
ranked_keywords = pd.DataFrame(data=ranked_text,columns=["Rank","Keyword"])
ranked_keywords.head()

,Rank,Keyword
0,25.000000,operators order highest lowest priority
1,24.000000,println width getparameter width system
2,24.000000,println p getparameter p system
3,21.000000,interpreted java compiler asnew string
4,17.909091,data new data b data


In [0]:
!pip install openpyxl
import openpyxl
ranked_keywords.to_excel('ranked_keywords.xlsx')
from google.colab import files
files.download('ranked_keywords.xlsx')